<a href="https://colab.research.google.com/github/tkkkkkz/Data_Science_Projects/blob/main/PM2.5_Sensor_Network/Outdoor_pair_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime, date
from scipy.stats import ttest_ind
from numpy import cos, sin, arcsin, sqrt
from math import radians

import warnings
warnings.filterwarnings('ignore')

In [34]:
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
link_single = '/content/gdrive/My Drive/Colab Notebooks/2017-SEPT 8 2021 single outdoor sensor 347687 9v.csv_DAILY.csv'
double_link = '/content/gdrive/My Drive/Colab Notebooks/Daily outdoor double sensor 2017-Sept 8 2021 3788769.csv'

In [4]:
single_daily = pd.read_csv(link_single)

In [5]:
single_daily ['Day'] = pd.to_datetime(single_daily['Day'])

In [6]:
single_sensor = single_daily.groupby(['sensor_index', 'longitude', 'latitude', 'County', 'State']).mean()
single_sensor.reset_index(inplace=True)
single_sensor = single_sensor[['sensor_index', 'longitude', 'latitude', 'County', 'State', 'Zip']]

In [7]:
double = pd.read_csv(double_link, usecols=range(20),sep=",", encoding='Latin-1')
double.head()

,sensor_index,name,run,channel_state,location_type,longitude,latitude,HouseNumber,Street,City,County,State,Zip,Country,Day,groupCount,MeanPM2.5A,MeanPM2.5B,Mean PM2.5 CF3,precision
0,185,CCA Bonnie Brae and Clinton,LosAngelesCounty_2017_RUN_2021-09-15_054524,3,0,-118.264,34.073,1765,Clinton Street,Los Angeles,Los Angeles County,California,90026,United States,42736,24,7.07601,9.919931,8.497970,0.167330
1,185,CCA Bonnie Brae and Clinton,LosAngelesCounty_2017_RUN_2021-09-15_054524,3,0,-118.264,34.073,1765,Clinton Street,Los Angeles,Los Angeles County,California,90026,United States,42737,24,2.86273,3.966498,3.414612,0.161625
2,185,CCA Bonnie Brae and Clinton,LosAngelesCounty_2017_RUN_2021-09-15_054524,3,0,-118.264,34.073,1765,Clinton Street,Los Angeles,Los Angeles County,California,90026,United States,42738,24,5.93947,8.125174,7.032322,0.155404
3,185,CCA Bonnie Brae and Clinton,LosAngelesCounty_2017_RUN_2021-09-15_054524,3,0,-118.264,34.073,1765,Clinton Street,Los Angeles,Los Angeles County,California,90026,United States,42739,24,8.5256,11.612003,10.068802,0.153266
4,185,CCA Bonnie Brae and Clinton,LosAngelesCounty_2017_RUN_2021-09-15_054524,3,0,-118.264,34.073,1765,Clinton Street,Los Angeles,Los Angeles County,California,90026,United States,42740,24,4.46974,6.234136,5.351937,0.164837


In [8]:
double['Day'] = pd.to_numeric(double['Day'],errors='coerce')
double['Day'] = pd.to_timedelta(double['Day'],unit='D') + pd.to_datetime('1899-12-30')

In [9]:
double_sensor = double.groupby(['sensor_index', 'longitude', 'latitude', 'County', 'State']).mean()
double_sensor.reset_index(inplace=True)
double_sensor = double_sensor[['sensor_index', 'longitude', 'latitude', 'County', 'State']]

In [10]:
single_sensor['key'] = 1
double_sensor['key'] = 1

In [11]:
single_double = single_sensor.merge(double_sensor, on='key',suffixes=('_s', '_d'))

In [12]:
single_double.head()

,sensor_index_s,longitude_s,latitude_s,County_s,State_s,Zip,key,sensor_index_d,longitude_d,latitude_d,County_d,State_d
0,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,185,-118.264,34.0730,Los Angeles County,California
1,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,187,-115.939,33.5400,Riverside County,California
2,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,193,-116.208,33.7003,Riverside County,California
3,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,195,-124.129,41.0597,Humboldt County,California
4,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,200,-116.076,33.5729,Riverside County,California


In [13]:
single_double.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438856 entries, 0 to 438855
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   sensor_index_s  438856 non-null  int64  
 1   longitude_s     438856 non-null  float64
 2   latitude_s      438856 non-null  float64
 3   County_s        438856 non-null  object 
 4   State_s         438856 non-null  object 
 5   Zip             438856 non-null  float64
 6   key             438856 non-null  int64  
 7   sensor_index_d  438856 non-null  int64  
 8   longitude_d     438856 non-null  object 
 9   latitude_d      438856 non-null  float64
 10  County_d        438856 non-null  object 
 11  State_d         438856 non-null  object 
dtypes: float64(4), int64(3), object(5)
memory usage: 43.5+ MB


In [14]:
single_double['longitude_d'] = pd.to_numeric(single_double['longitude_d'],errors='coerce')

In [15]:
single_double.dropna(inplace=True)

In [16]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [17]:
single_double['distance'] = haversine_np(single_double['longitude_s'],single_double['latitude_s'],single_double['longitude_d'], single_double['latitude_d'])

In [18]:
single_double.head()

,sensor_index_s,longitude_s,latitude_s,County_s,State_s,Zip,key,sensor_index_d,longitude_d,latitude_d,County_d,State_d,distance
0,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,185,-118.2640,34.0730,Los Angeles County,California,482.835224
1,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,187,-115.9388,33.5400,Riverside County,California,679.411911
2,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,193,-116.2077,33.7003,Riverside County,California,648.874842
3,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,195,-124.1289,41.0597,Humboldt County,California,463.026912
4,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,200,-116.0760,33.5729,Riverside County,California,667.283178


In [19]:
paired = single_double[single_double['distance']<=1]

In [20]:
paired.head(50)

,sensor_index_s,longitude_s,latitude_s,County_s,State_s,Zip,key,sensor_index_d,longitude_d,latitude_d,County_d,State_d,distance
4578,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,53455,-121.8856,37.2826,Santa Clara County,California,0.989124
5595,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,63619,-121.8819,37.2738,Santa Clara County,California,0.044450
6345,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,68291,-121.8883,37.2753,Santa Clara County,California,0.578982
6597,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,69353,-121.8893,37.2725,Santa Clara County,California,0.681093
6804,100,-121.8819,37.2742,Santa Clara County,California,95118.0,1,70095,-121.8850,37.2770,Santa Clara County,California,0.414676
10107,129,-122.5080,37.7582,San Francisco County,California,94122.0,1,1742,-122.5077,37.7581,San Francisco County,California,0.028604
12332,129,-122.5080,37.7582,San Francisco County,California,94122.0,1,19159,-122.5029,37.7636,San Francisco County,California,0.748891
12333,129,-122.5080,37.7582,San Francisco County,California,94122.0,1,19163,-122.5013,37.7603,San Francisco County,California,0.633196
12895,129,-122.5080,37.7582,San Francisco County,California,94122.0,1,22589,-122.5055,37.7536,San Francisco County,California,0.556368
13047,129,-122.5080,37.7582,San Francisco County,California,94122.0,1,25187,-122.5012,37.7604,San Francisco County,California,0.645497


In [21]:
paired.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 4578 to 438103
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sensor_index_s  212 non-null    int64  
 1   longitude_s     212 non-null    float64
 2   latitude_s      212 non-null    float64
 3   County_s        212 non-null    object 
 4   State_s         212 non-null    object 
 5   Zip             212 non-null    float64
 6   key             212 non-null    int64  
 7   sensor_index_d  212 non-null    int64  
 8   longitude_d     212 non-null    float64
 9   latitude_d      212 non-null    float64
 10  County_d        212 non-null    object 
 11  State_d         212 non-null    object 
 12  distance        212 non-null    float64
dtypes: float64(6), int64(3), object(4)
memory usage: 23.2+ KB


In [65]:
 # to get an empty table
single_one = single_daily[single_daily['sensor_index'] == 100]
double_one = double[double['sensor_index'] == 69353]
all_tables = pd.merge(single_one, double_one, on='Day',suffixes=('_s', '_d'))
all_tables

,sensor_index_s,name_s,longitude_s,latitude_s,County_s,State_s,Zip_s,Day,groupCount_s,PM2.5_CF3_A,sensor_index_d,name_d,run,channel_state,location_type,longitude_d,latitude_d,HouseNumber,Street,City,County_d,State_d,Zip_d,Country,groupCount_d,MeanPM2.5A,MeanPM2.5B,Mean PM2.5 CF3,precision


In [74]:
sensor_id_s = []
sensor_id_d = []
n = []
min_date = []
max_date = []

County_s = []
County_d = []
longitude_s = []
latitude_s = []
longitude_d = []
latitude_d = []

PM25_min_s = []
PM25_25_s = []
PM25_mean_s = []
PM25_median_s = []
PM25_75_s = []
PM25_max_s = []

PM25_min_d = []
PM25_25_d = []
PM25_mean_d = []
PM25_median_d = []
PM25_75_d = []
PM25_max_d = []

intercept_summary = []
slope_summary = []
r_value_summary = []
p_value_summary = []
slope_std_summary = []


for s_index, d_index in paired[['sensor_index_s', 'sensor_index_d']].itertuples(index=False):
  single_one = single_daily[single_daily['sensor_index'] == s_index]
  double_one = double[double['sensor_index'] == d_index]
  merge = pd.merge(single_one, double_one, on='Day',suffixes=('_s', '_d'))
  if len(merge) > 0:
    file_name = 'paired_'+ str(s_index) + '_' + str(d_index) +'.csv'
    #merge.to_csv('gdrive/MyDrive/Paired/' + file_name)
    #print(file_name)
    sensor_id_s.append(merge['sensor_index_s'][0])
    sensor_id_d.append(merge['sensor_index_d'][0])
    n.append(len(merge))
    min_date.append(min(merge['Day']))
    max_date.append(max(merge['Day']))

    County_s.append(merge['County_s'][0])
    County_d.append(merge['County_d'][0])
    longitude_s.append(merge['longitude_s'][0])
    latitude_s.append(merge['latitude_s'][0])
    longitude_d.append(merge['longitude_d'][0])
    latitude_d.append(merge['latitude_d'][0])

    PM25_min_s.append(merge['PM2.5_CF3_A'].min())
    PM25_25_s.append(merge['PM2.5_CF3_A'].quantile(0.25))
    PM25_mean_s.append(round(merge['PM2.5_CF3_A'].mean(), 2))
    PM25_median_s.append(merge['PM2.5_CF3_A'].median()) 
    PM25_75_s.append(merge['PM2.5_CF3_A'].quantile(0.75))
    PM25_max_s.append(merge['PM2.5_CF3_A'].max()) 

    PM25_min_d.append(merge['Mean PM2.5 CF3'].min())
    PM25_25_d.append(merge['Mean PM2.5 CF3'].quantile(0.25))
    PM25_mean_d.append(round(merge['Mean PM2.5 CF3'].mean(), 2))
    PM25_median_d.append(merge['Mean PM2.5 CF3'].median())
    PM25_75_d.append(merge['Mean PM2.5 CF3'].quantile(0.75))
    PM25_max_d.append(merge['Mean PM2.5 CF3'].max())

    slope, intercept, r_value, p_value, slope_std = stats.linregress(merge['PM2.5_CF3_A'],merge['Mean PM2.5 CF3'],)
    intercept_summary.append(intercept)
    slope_summary.append(slope)
    r_value_summary.append(r_value)
    p_value_summary.append(p_value)
    slope_std_summary.append(slope_std)

    #all_tables = pd.concat([all_tables, merge],axis=0)


In [75]:
stats_summary = pd.DataFrame({'sensor_id_s': sensor_id_s, 'sensor_id_d': sensor_id_d, 'n':n,'min_date':min_date,'max_date':max_date,'County_s':County_s,
'County_d':County_d,'longitude_s': longitude_s,'latitude_s':latitude_s,'longitude_d': longitude_d,'latitude_d':latitude_d, 'PM2.5_min_s':PM25_min_s,
'PM2.5_25_s':PM25_25_s , 'PM2.5_mean_s': PM25_mean_s, 'PM2.5_median_s': PM25_median_s,'PM2.5_75_s':PM25_75_s,'PM2.5_max_s':PM25_max_s, 'PM2.5_min_d':PM25_min_d,
'PM2.5_25_d':PM25_25_d, 'PM2.5_mean_d':PM25_mean_d,'PM2.5_median_d':PM25_median_d,'PM2.5_75_d':PM25_75_d,'PM2.5_max_d':PM25_max_d,
'intercept_summary':intercept_summary, 'slope_summary':slope_summary,'r_value_summary':r_value_summary,'p_value_summary': p_value_summary, 'slope_std_summary':slope_std_summary})

stats_summary.to_csv('gdrive/MyDrive/Paired/stats_summary.csv')

In [68]:
all_tables.to_csv('gdrive/MyDrive/Paired/whole_paird_outdoor.csv')